In [ ]:
import cv2
import os
from IPython.display import display, Javascript # display và Javascript được dùng trong hàm video_stream
from google.colab.output import eval_js # eval_js được dùng trong hàm video_frame
import PIL.Image # Để làm việc với ảnh, đặc biệt là chuyển đổi RGBA sang PNG
import numpy as np
import base64
import io # Để làm việc với byte stream
import time
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Đoạn mã này có tác dụng chính là tải xuống và khởi tạo các bộ phân loại Haar Cascade được huấn luyện sẵn của OpenCV để phát hiện khuôn mặt và mắt (hỏi)*.*`

In [ ]:
HAAR_FACE_URL = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
HAAR_EYE_URL = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml"
HAAR_FACE_FILE = "haarcascade_frontalface_default.xml"
HAAR_EYE_FILE = "haarcascade_eye.xml"

def download_file(url, filename):
    if not os.path.exists(filename):
        print(f"Đang tải {filename} từ {url}...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192): f.write(chunk)
            print(f"Đã tải xong {filename}")
        except requests.exceptions.RequestException as e:
            print(f"Lỗi khi tải {filename}: {e}")
            return False
    else:
        print(f"{filename} đã tồn tại.")
    return True

print("Đang kiểm tra và tải các tệp Haar Cascade...")
face_cascade_loaded = False
eye_cascade_loaded = False

if download_file(HAAR_FACE_URL, HAAR_FACE_FILE):
    face_cascade = cv2.CascadeClassifier(HAAR_FACE_FILE)
    if face_cascade.empty():
        print(f"LỖI: Không thể tải bộ phân loại khuôn mặt từ tệp {HAAR_FACE_FILE}.")
    else:
        print("Bộ phân loại khuôn mặt đã được tải thành công.")
        face_cascade_loaded = True

if download_file(HAAR_EYE_URL, HAAR_EYE_FILE):
    eye_cascade = cv2.CascadeClassifier(HAAR_EYE_FILE)
    if eye_cascade.empty():
        print(f"LỖI: Không thể tải bộ phân loại mắt từ tệp {HAAR_EYE_FILE}.")
    else:
        print("Bộ phân loại mắt đã được tải thành công.")
        eye_cascade_loaded = True


Đang kiểm tra và tải các tệp Haar Cascade...
Đang tải haarcascade_frontalface_default.xml từ https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml...


NameError: name 'requests' is not defined

# *Hai hàm tiện ích để chuyển đổi dữ liệu hình ảnh giữa các định dạng khác nhau, thường được sử dụng trong các ứng dụng web hoặc khi làm việc với hình ảnh được truyền dưới dạng chuỗi (hỏi):*

In [ ]:
def js_to_image(js_reply_img):
    """
    Chuyển đổi một ảnh Javascript (định dạng Data URL base64) sang ảnh OpenCV BGR.
    js_reply_img: chuỗi base64 data URL của ảnh.
    """
    image_bytes = base64.b64decode(js_reply_img.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=cv2.IMREAD_COLOR)
    return img

def bbox_to_bytes(bbox_array):
    """
    Chuyển đổi một mảng NumPy (RGBA) chứa bounding box thành chuỗi Data URL PNG base64.
    bbox_array: mảng NumPy RGBA.
    """
    # Tạo ảnh PIL từ mảng NumPy (phải là RGBA để có kênh alpha)
    img_pil = PIL.Image.fromarray(bbox_array, 'RGBA')
    buffer = io.BytesIO()
    # Lưu ảnh PIL vào buffer dưới dạng PNG (để giữ độ trong suốt)
    img_pil.save(buffer, format='PNG')
    # Lấy dữ liệu byte
    byte_data = buffer.getvalue()
    # Mã hóa sang base64
    base64_str = base64.b64encode(byte_data).decode('utf-8')
    # Trả về dưới dạng Data URL
    return 'data:image/png;base64,' + base64_str

print("\nCác hàm và tệp Haar Cascade đã sẵn sàng.")


Các hàm và tệp Haar Cascade đã sẵn sàng.


In [ ]:
# (Các import cần thiết nếu bạn chạy ô này riêng lẻ, nhưng giả sử Ô 1 đã chạy)
# import cv2
# import os
# from IPython.display import display, Javascript
# from google.colab.output import eval_js
# import PIL.Image
# import numpy as np
# import base64
# import io
# import time

# --- 1. Thiết lập thư mục lưu trữ khuôn mặt ---
# Chọn một trong các đường dẫn bạn đã tạo ở ô trước.
# Ví dụ: Lưu vào thư mục tạm thời của Colab:
SAVE_FOLDER_PATH = "/content/khuon_mat_da_luu_colab"
# Hoặc nếu bạn muốn lưu vào Google Drive (đảm bảo Drive đã được mount và thư mục tồn tại):
# GDRIVE_MOUNT_PATH = "/content/drive"
# GDRIVE_FOLDER_NAME = "KhuonMatDaLuuTrenDrive" # Tên thư mục bạn đã tạo trên Drive
# SAVE_FOLDER_PATH = os.path.join(GDRIVE_MOUNT_PATH, "MyDrive", GDRIVE_FOLDER_NAME)

# Đảm bảo thư mục tồn tại (tạo nếu chưa có)
if not os.path.exists(SAVE_FOLDER_PATH):
    os.makedirs(SAVE_FOLDER_PATH, exist_ok=True)
    print(f"Đã tạo thư mục lưu trữ: {SAVE_FOLDER_PATH}")
else:
    print(f"Sử dụng thư mục lưu trữ hiện có: {SAVE_FOLDER_PATH}")

# Kiểm tra lại xem các bộ phân loại đã được tải chưa
if not 'face_cascade_loaded' in globals() or not 'eye_cascade_loaded' in globals() or \
   not face_cascade_loaded or not eye_cascade_loaded:
    print("LỖI: Các bộ phân loại Haar Cascade chưa được tải. Vui lòng chạy Ô 1 trước và đảm bảo không có lỗi.")
else:
    # --- 2. JavaScript và Python functions từ mã của bạn (giữ nguyên như trước) ---
    def video_stream():
      js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement; // Để hiển thị overlay
        var labelElement; // Để hiển thị status/label

        var pendingResolve = null;
        var shutdown = false;

        function removeDom() {
          if (stream) {
            stream.getVideoTracks().forEach(track => track.stop());
          }
          if (video) video.remove();
          if (div) div.remove();
          video = null;
          div = null;
          stream = null;
          imgElement = null;
          captureCanvas = null;
          labelElement = null;
        }

        function onAnimationFrame() {
          if (!shutdown) {
            window.requestAnimationFrame(onAnimationFrame);
          }
          if (pendingResolve) {
            var result = "";
            if (!shutdown && video && video.readyState === 4 && video.videoWidth > 0) {
              captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
              result = captureCanvas.toDataURL('image/jpeg', 0.8);
            }
            var lp = pendingResolve;
            pendingResolve = null;
            lp(result);
          }
        }

        async function createDom() {
          if (div !== null) {
            return stream;
          }
          shutdown = false;
          div = document.createElement('div');
          div.style.border = '2px solid black';
          div.style.padding = '3px';
          div.style.width = '100%';
          div.style.maxWidth = '610px';
          document.body.appendChild(div);

          const modelOut = document.createElement('div');
          modelOut.innerHTML = "<span>Trạng thái: </span>";
          labelElement = document.createElement('span');
          labelElement.innerText = 'Đang khởi tạo...';
          labelElement.style.fontWeight = 'bold';
          modelOut.appendChild(labelElement);
          div.appendChild(modelOut);

          video = document.createElement('video');
          video.style.display = 'block';
          video.width = 600;
          video.height = 450;
          video.setAttribute('playsinline', '');
          video.onclick = () => { shutdown = true; };

          stream = await navigator.mediaDevices.getUserMedia(
              {video: { facingMode: "user", width: 640, height: 480 }});
          div.appendChild(video);

          imgElement = document.createElement('img');
          imgElement.style.position = 'absolute';
          imgElement.style.zIndex = 1;
          div.appendChild(imgElement);

          const instruction = document.createElement('div');
          instruction.innerHTML =
              '<span style="color: red; font-weight: bold;">' +
              'Khi hoàn tất, nhấp vào video hoặc dòng chữ này để dừng.</span>';
          div.appendChild(instruction);
          instruction.onclick = () => { shutdown = true; };

          video.srcObject = stream;
          await video.play();

          captureCanvas = document.createElement('canvas');
          captureCanvas.width = 640;
          captureCanvas.height = 480;
          window.requestAnimationFrame(onAnimationFrame);

          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
          labelElement.innerText = 'Sẵn sàng!';
          return stream;
        }

        async function stream_frame(label, imgData) {
          if (shutdown) {
            removeDom();
            return '';
          }
          var preCreate = Date.now();
          stream = await createDom();
          if (!stream) {
              removeDom();
              return '';
          }
          var preShow = Date.now();
          if (labelElement && label != "") {
            labelElement.innerHTML = label;
          }
          if (imgElement && imgData != "" && video) {
            var videoRect = video.getBoundingClientRect();
            imgElement.style.top = videoRect.top + "px";
            imgElement.style.left = videoRect.left + "px";
            imgElement.style.width = videoRect.width + "px";
            imgElement.style.height = videoRect.height + "px";
            imgElement.src = imgData;
          }
          var preCapture = Date.now();
          var result = await new Promise(function(resolve, reject) {
            pendingResolve = resolve;
          });
          if (result) {
            return {'create': preShow - preCreate,
                    'show': preCapture - preShow,
                    'capture': Date.now() - preCapture,
                    'img': result};
          } else {
            removeDom();
            return '';
          }
        }
        ''')
      display(js)

    def video_frame(label, bbox_data_url):
      data = eval_js('stream_frame("{}", "{}")'.format(label, bbox_data_url))
      return data

    # --- 3. Bắt đầu luồng video, xử lý và LƯU KHUÔN MẶT ---
    print("Đang khởi tạo luồng video từ webcam...")
    print("Trình duyệt sẽ yêu cầu quyền truy cập camera. Vui lòng đồng ý.")
    video_stream()

    label_html = 'Đang chụp...'
    bbox_data_url = ''
    count = 0
    saved_face_count = 0 # Biến đếm số khuôn mặt đã lưu
    max_frames = 500

    try:
        while count < max_frames:
            js_reply = video_frame(label_html, bbox_data_url)

            if not js_reply or not js_reply.get("img"):
                print("Luồng video đã dừng hoặc không nhận được ảnh.")
                break

            img_bgr = js_to_image(js_reply["img"])

            if img_bgr is None:
                print("Lỗi: không thể decode ảnh từ JS.")
                time.sleep(0.5)
                continue

            bbox_overlay_rgba = np.zeros((480, 640, 4), dtype=np.uint8)
            gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.15, minNeighbors=5, minSize=(40,40)) # Tăng minSize

            face_detected_this_frame = False
            for (x, y, w, h) in faces:
                face_detected_this_frame = True
                # Vẽ bounding box cho khuôn mặt lên overlay
                cv2.rectangle(bbox_overlay_rgba, (x, y), (x + w, y + h), (255, 0, 0, 255), 2)

                # ---- PHẦN LƯU KHUÔN MẶT ----
                # Cắt vùng khuôn mặt từ ảnh gốc (ảnh màu BGR)
                face_roi = img_bgr[y:y+h, x:x+w]

                # Đảm bảo vùng ROI không rỗng trước khi lưu
                if face_roi.size > 0:
                    # Tạo tên file duy nhất
                    timestamp = int(time.time() * 1000) # Milliseconds timestamp
                    filename = os.path.join(SAVE_FOLDER_PATH, f"face_{timestamp}_{saved_face_count:04d}.jpg")
                    # Lưu file ảnh khuôn mặt
                    cv2.imwrite(filename, face_roi)
                    print(f"Đã lưu khuôn mặt vào: {filename}")
                    saved_face_count += 1
                # -----------------------------

                roi_gray_face = gray[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(roi_gray_face, scaleFactor=1.1, minNeighbors=4, minSize=(20,20)) # Tăng minSize cho mắt
                for (ex, ey, ew, eh) in eyes:
                    cv2.rectangle(bbox_overlay_rgba, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (0, 255, 0, 255), 1)

            bbox_overlay_rgba[:, :, 3] = (bbox_overlay_rgba.max(axis=2) > 0).astype(int) * 255
            bbox_data_url = bbox_to_bytes(bbox_overlay_rgba)

            if face_detected_this_frame:
                label_html = f"Đã phát hiện {len(faces)} khuôn mặt."
            else:
                label_html = "Không phát hiện khuôn mặt."

            count += 1
            # Bỏ print timing để output đỡ rối khi có print lưu file
            # print(f"Frame {count}: {label_html} | Timings (ms): Create: {js_reply.get('create',0)}, Show: {js_reply.get('show',0)}, Capture: {js_reply.get('capture',0)}")
            print(f"Frame {count}: {label_html}")


    except Exception as e:
        if 'NotFoundError' in str(e) or 'NotAllowedError' in str(e):
             print("Lỗi truy cập camera: Có thể bạn chưa cấp quyền hoặc không tìm thấy camera.")
        else:
            print(f"Đã xảy ra lỗi trong vòng lặp Python: {e}")
    finally:
        try:
            eval_js('removeDom()')
            print("Đã yêu cầu JavaScript dọn dẹp DOM.")
        except Exception as e_cleanup:
            print(f"Lỗi khi cố gắng dọn dẹp JavaScript DOM: {e_cleanup}")
        print(f"Đã xử lý {count} khung hình. Đã lưu tổng cộng {saved_face_count} khuôn mặt vào '{SAVE_FOLDER_PATH}'.")
        print("Chương trình kết thúc.")


Đã tạo thư mục lưu trữ: /content/khuon_mat_da_luu_colab
LỖI: Các bộ phân loại Haar Cascade chưa được tải. Vui lòng chạy Ô 1 trước và đảm bảo không có lỗi.


In [ ]:
# -*- coding: utf-8 -*-
"""
Tối ưu hóa code nhận diện và lưu khuôn mặt từ Webcam trên Colab.
"""

# --- 1. Imports ---
import cv2
import os
import time
import requests # Để tải file
import base64
import io # Để làm việc với byte stream
import numpy as np
import PIL.Image # Để làm việc với ảnh, đặc biệt là chuyển đổi RGBA sang PNG
from IPython.display import display, Javascript # display và Javascript được dùng trong hàm video_stream
from google.colab.output import eval_js # eval_js được dùng trong hàm video_frame
from google.colab import drive # Để tùy chọn lưu vào Google Drive

# --- 2. Constants ---
# URLs và tên file cho Haar Cascades
HAAR_FACE_URL = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
HAAR_EYE_URL = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml"
HAAR_FACE_FILE = "haarcascade_frontalface_default.xml"
HAAR_EYE_FILE = "haarcascade_eye.xml"

# --- 3. Google Drive Mount (Tùy chọn) ---
# Nếu bạn muốn lưu vào Google Drive, hãy bỏ comment dòng này và chạy ô này trước
# drive.mount('/content/drive')
GDRIVE_MOUNT_PATH = "/content/drive" # Đường dẫn mount mặc định của Colab

# --- 4. Configuration / Cấu hình ---
# Chọn thư mục lưu trữ:
# Lựa chọn 1: Thư mục tạm thời của Colab (sẽ mất khi runtime reset)
SAVE_FOLDER_PATH = "/content/khuon_mat_da_luu_colab"

# Lựa chọn 2: Thư mục trên Google Drive (yêu cầu mount ở trên)
# GDRIVE_FOLDER_NAME = "KhuonMatDaLuuTrenDrive" # Tên thư mục bạn muốn tạo/sử dụng trên Drive
# SAVE_FOLDER_PATH = os.path.join(GDRIVE_MOUNT_PATH, "MyDrive", GDRIVE_FOLDER_NAME)

# Khoảng cách giữa các lần lưu khuôn mặt (ví dụ: 10 nghĩa là lưu mỗi 10 frame có phát hiện khuôn mặt)
# Đặt là 1 nếu muốn lưu mọi frame có khuôn mặt (như code gốc)
SAVE_INTERVAL = 10

# Số khung hình tối đa xử lý trước khi tự động dừng
MAX_FRAMES_TO_PROCESS = 500

# Tham số cho bộ phát hiện khuôn mặt Haar Cascade
# scaleFactor: Giảm kích thước ảnh ở mỗi tỷ lệ. Giá trị > 1. Giá trị nhỏ hơn (vd: 1.05) chậm hơn nhưng có thể phát hiện nhiều khuôn mặt hơn.
# minNeighbors: Số lượng hàng xóm mà mỗi hình chữ nhật ứng viên nên có để giữ lại nó. Giá trị cao hơn -> ít phát hiện hơn nhưng chất lượng cao hơn.
# minSize: Kích thước đối tượng tối thiểu. Các đối tượng nhỏ hơn sẽ bị bỏ qua.
FACE_SCALE_FACTOR = 1.15
FACE_MIN_NEIGHBORS = 6 # Tăng nhẹ để giảm false positives
FACE_MIN_SIZE = (40, 40)

# Tham số cho bộ phát hiện mắt Haar Cascade
EYE_SCALE_FACTOR = 1.1
EYE_MIN_NEIGHBORS = 4
EYE_MIN_SIZE = (25, 25) # Tăng nhẹ

# --- 5. Helper Functions ---
def download_file(url, filename):
    """Tải file từ URL nếu nó chưa tồn tại."""
    if not os.path.exists(filename):
        print(f"Đang tải {filename} từ {url}...")
        try:
            response = requests.get(url, stream=True, timeout=30) # Thêm timeout
            response.raise_for_status() # Kiểm tra lỗi HTTP
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Đã tải xong {filename}")
            return True
        except requests.exceptions.RequestException as e:
            print(f"LỖI khi tải {filename}: {e}")
            return False
        except Exception as e:
            print(f"LỖI không xác định khi tải {filename}: {e}")
            return False
    else:
        print(f"{filename} đã tồn tại.")
        return True

def js_to_image(js_reply_img):
    """Chuyển đổi ảnh Javascript (base64 data URL) sang ảnh OpenCV BGR."""
    try:
        # Tách phần dữ liệu base64
        image_data = js_reply_img.split(',')[1]
        # Decode base64
        image_bytes = base64.b64decode(image_data)
        # Chuyển byte thành mảng NumPy
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        # Decode mảng NumPy thành ảnh OpenCV
        img = cv2.imdecode(jpg_as_np, flags=cv2.IMREAD_COLOR)
        return img
    except IndexError:
        print("LỖI js_to_image: Định dạng data URL không hợp lệ.")
        return None
    except base64.binascii.Error:
        print("LỖI js_to_image: Lỗi giải mã Base64.")
        return None
    except Exception as e:
        print(f"LỖI js_to_image: Lỗi không xác định: {e}")
        return None

def bbox_to_bytes(bbox_array):
    """Chuyển đổi mảng NumPy RGBA thành chuỗi Data URL PNG base64."""
    try:
        # Tạo ảnh PIL từ mảng NumPy (phải là RGBA)
        img_pil = PIL.Image.fromarray(bbox_array, 'RGBA')
        buffer = io.BytesIO()
        # Lưu ảnh PIL vào buffer dạng PNG để giữ độ trong suốt
        img_pil.save(buffer, format='PNG')
        # Lấy dữ liệu byte
        byte_data = buffer.getvalue()
        # Mã hóa sang base64
        base64_str = base64.b64encode(byte_data).decode('utf-8')
        # Trả về dạng Data URL
        return 'data:image/png;base64,' + base64_str
    except Exception as e:
        print(f"LỖI bbox_to_bytes: {e}")
        return "" # Trả về chuỗi rỗng nếu có lỗi

# --- 6. JavaScript Functions for Webcam Interaction ---
def video_stream():
    """Hiển thị luồng video và các thành phần điều khiển bằng JavaScript."""
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement; // Để hiển thị overlay
        var labelElement; // Để hiển thị status/label

        var pendingResolve = null;
        var shutdown = false;

        // Hàm dọn dẹp các phần tử DOM
        function removeDom() {
            shutdown = true; // Đảm bảo dừng vòng lặp animation frame
            if (stream) {
                stream.getVideoTracks().forEach(track => track.stop());
            }
            if (video) video.remove();
            if (div) div.remove();
            video = null; div = null; stream = null; imgElement = null;
            captureCanvas = null; labelElement = null; pendingResolve = null;
            console.log("DOM elements removed.");
        }

        // Vòng lặp xử lý frame
        function onAnimationFrame() {
            if (!shutdown) {
                window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
                var result = "";
                // Chỉ chụp khi video sẵn sàng và có kích thước hợp lệ
                if (!shutdown && video && video.readyState === 4 && video.videoWidth > 0) {
                    try {
                        captureCanvas.width = video.videoWidth; // Cập nhật kích thước canvas nếu cần
                        captureCanvas.height = video.videoHeight;
                        captureCanvas.getContext('2d').drawImage(video, 0, 0, video.videoWidth, video.videoHeight);
                        result = captureCanvas.toDataURL('image/jpeg', 0.8); // Chất lượng JPEG 80%
                    } catch (e) {
                        console.error("Error drawing video to canvas:", e);
                        result = ""; // Trả về rỗng nếu có lỗi vẽ
                    }
                }
                var lp = pendingResolve;
                pendingResolve = null;
                lp(result); // Gửi kết quả (ảnh base64 hoặc chuỗi rỗng) về Python
            }
        }

        // Tạo các phần tử DOM và khởi tạo stream
        async function createDom() {
            if (div !== null) { return stream; } // Tránh tạo lại nếu đã tồn tại

            shutdown = false;
            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '610px'; // Giới hạn chiều rộng tối đa
            document.body.appendChild(div);

            // Hiển thị trạng thái
            const modelOut = document.createElement('div');
            modelOut.innerHTML = "<span>Trạng thái: </span>";
            labelElement = document.createElement('span');
            labelElement.innerText = 'Đang khởi tạo...';
            labelElement.style.fontWeight = 'bold';
            modelOut.appendChild(labelElement);
            div.appendChild(modelOut);

            // Phần tử Video
            video = document.createElement('video');
            video.style.display = 'block';
            video.width = 600; // Đặt kích thước hiển thị mong muốn
            video.height = 450;
            video.setAttribute('playsinline', ''); // Quan trọng cho mobile
            video.autoplay = true; // Tự động phát khi có stream
            video.muted = true; // Tắt tiếng để tránh lỗi autoplay policy
            video.onclick = () => { removeDom(); }; // Dừng khi click video

            // Phần tử Ảnh Overlay (cho bounding box)
            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute'; // Đặt chồng lên video
            imgElement.style.zIndex = 1; // Đảm bảo nằm trên video
            // Kích thước và vị trí sẽ được cập nhật trong stream_frame

            // Canvas ẩn để chụp ảnh từ video
            captureCanvas = document.createElement('canvas');
            captureCanvas.style.display = 'none'; // Không cần hiển thị

            // Hướng dẫn và nút dừng
            const instruction = document.createElement('div');
            instruction.innerHTML =
                '<span style="color: red; font-weight: bold;">' +
                'Nhấp vào video hoặc dòng chữ này để dừng.</span>';
            instruction.onclick = () => { removeDom(); };

            // Gắn các phần tử vào div chính
            div.appendChild(video);
            div.appendChild(imgElement); // Gắn overlay vào *sau* video trong DOM
            div.appendChild(instruction);

            // Yêu cầu truy cập camera
            try {
                stream = await navigator.mediaDevices.getUserMedia(
                    { video: { facingMode: "user", width: 640, height: 480 } } // Yêu cầu độ phân giải cụ thể
                );
                video.srcObject = stream;
            } catch (e) {
                console.error("Lỗi khi truy cập camera:", e);
                labelElement.innerText = 'Lỗi camera! Kiểm tra quyền hoặc kết nối.';
                removeDom(); // Dọn dẹp nếu không truy cập được camera
                return null;
            }

            // Chờ video sẵn sàng
            await video.play();

            // Điều chỉnh chiều cao iframe Colab
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Bắt đầu vòng lặp animation frame
            window.requestAnimationFrame(onAnimationFrame);

            labelElement.innerText = 'Sẵn sàng!';
            return stream;
        }

        // Hàm được gọi từ Python để gửi/nhận frame
        async function stream_frame(label, imgData) {
            if (shutdown) { return ''; } // Nếu đã yêu cầu dừng, trả về rỗng

            var preCreate = Date.now();
            stream = await createDom(); // Tạo DOM nếu chưa có
            if (!stream) { return ''; } // Nếu không tạo được stream, dừng
            var preShow = Date.now();

            // Cập nhật label trạng thái
            if (labelElement && label != "") { labelElement.innerHTML = label; }

            // Cập nhật ảnh overlay (bounding box)
            if (imgElement && imgData != "" && video && video.getBoundingClientRect) {
                // Lấy kích thước và vị trí thực tế của video trên màn hình
                var videoRect = video.getBoundingClientRect();
                imgElement.style.top = videoRect.top + "px";
                imgElement.style.left = videoRect.left + "px";
                imgElement.style.width = videoRect.width + "px";
                imgElement.style.height = videoRect.height + "px";
                imgElement.src = imgData; // Đặt nguồn ảnh overlay
            }

            var preCapture = Date.now();
            // Chờ promise được resolve bởi onAnimationFrame để lấy ảnh mới
            var result = await new Promise(function(resolve, reject) {
                pendingResolve = resolve;
            });

            // Trả về dữ liệu ảnh và thời gian xử lý (nếu thành công)
            if (result) {
                return {'create': preShow - preCreate,
                        'show': preCapture - preShow,
                        'capture': Date.now() - preCapture,
                        'img': result};
            } else {
                // Nếu không nhận được ảnh (ví dụ: stream dừng giữa chừng)
                removeDom(); // Gọi dọn dẹp từ đây
                return '';
            }
        }
        ''')
    display(js)

def video_frame(label, bbox_data_url):
    """Gọi hàm JavaScript stream_frame và trả về kết quả."""
    try:
        data = eval_js('stream_frame("{}", "{}")'.format(label, bbox_data_url))
        return data
    except Exception as e:
        # Có thể xảy ra lỗi nếu iframe bị đóng đột ngột
        print(f"Lỗi khi gọi eval_js('stream_frame'): {e}")
        return None # Trả về None để vòng lặp Python biết dừng

# --- 7. Main Execution ---
def run_face_detection_and_save():
    """Hàm chính để chạy toàn bộ quá trình."""
    print("--- Khởi tạo ---")

    # Đảm bảo thư mục lưu trữ tồn tại
    try:
        if not os.path.exists(SAVE_FOLDER_PATH):
            os.makedirs(SAVE_FOLDER_PATH, exist_ok=True)
            print(f"Đã tạo thư mục lưu trữ: {SAVE_FOLDER_PATH}")
        else:
            print(f"Sử dụng thư mục lưu trữ hiện có: {SAVE_FOLDER_PATH}")
    except OSError as e:
        print(f"LỖI: Không thể tạo hoặc truy cập thư mục '{SAVE_FOLDER_PATH}': {e}")
        print("Vui lòng kiểm tra đường dẫn và quyền truy cập, đặc biệt nếu dùng Google Drive.")
        return # Dừng thực thi nếu không có thư mục lưu

    # Tải và kiểm tra Haar Cascades
    print("\nĐang kiểm tra và tải các tệp Haar Cascade...")
    face_cascade = None
    eye_cascade = None
    face_cascade_loaded = False
    eye_cascade_loaded = False

    if download_file(HAAR_FACE_URL, HAAR_FACE_FILE):
        face_cascade = cv2.CascadeClassifier(HAAR_FACE_FILE)
        if face_cascade.empty():
            print(f"LỖI: Không thể tải bộ phân loại khuôn mặt từ {HAAR_FACE_FILE}.")
        else:
            print("-> Bộ phân loại khuôn mặt đã tải thành công.")
            face_cascade_loaded = True

    if download_file(HAAR_EYE_URL, HAAR_EYE_FILE):
        eye_cascade = cv2.CascadeClassifier(HAAR_EYE_FILE)
        if eye_cascade.empty():
            print(f"LỖI: Không thể tải bộ phân loại mắt từ {HAAR_EYE_FILE}.")
        else:
            print("-> Bộ phân loại mắt đã tải thành công.")
            eye_cascade_loaded = True

    # Kiểm tra xem cả hai bộ phân loại đã sẵn sàng chưa
    if not face_cascade_loaded or not eye_cascade_loaded:
        print("\nLỖI: Không thể tiếp tục vì một hoặc cả hai bộ phân loại Haar Cascade chưa được tải.")
        print("Vui lòng kiểm tra lại kết nối mạng hoặc đường dẫn URL.")
        return # Dừng thực thi

    print("\n--- Bắt đầu Luồng Video và Nhận diện ---")
    print("Trình duyệt sẽ yêu cầu quyền truy cập camera. Vui lòng đồng ý.")
    video_stream() # Khởi tạo giao diện JavaScript

    label_html = 'Đang khởi tạo...'
    bbox_data_url = '' # Overlay ban đầu rỗng
    frame_count = 0
    saved_face_count = 0 # Đếm số khuôn mặt thực sự được lưu
    last_save_frame = -SAVE_INTERVAL # Để đảm bảo lần phát hiện đầu tiên được lưu (nếu SAVE_INTERVAL > 0)

    try:
        while frame_count < MAX_FRAMES_TO_PROCESS:
            # Lấy frame từ JavaScript
            js_reply = video_frame(label_html, bbox_data_url)

            # Kiểm tra xem stream có còn hoạt động hay không
            if not js_reply:
                print("\nThông báo: Luồng video đã dừng (có thể do người dùng nhấp dừng).")
                break
            if not js_reply.get("img"):
                print("\nCảnh báo: Không nhận được ảnh từ luồng video, có thể đang dừng...")
                time.sleep(0.5) # Chờ một chút xem có phải chỉ là tạm thời
                # Thử gọi lại một lần nữa để xác nhận dừng
                js_reply = video_frame(label_html, bbox_data_url)
                if not js_reply or not js_reply.get("img"):
                     print("Xác nhận luồng video đã dừng.")
                     break
                else:
                     print("...Luồng video đã tiếp tục.") # Nếu nhận lại được ảnh

            # Chuyển đổi ảnh từ base64 (JS) sang BGR (OpenCV)
            img_bgr = js_to_image(js_reply["img"])
            if img_bgr is None:
                print("Cảnh báo: Không thể decode ảnh từ JS, bỏ qua frame này.")
                time.sleep(0.1) # Nghỉ ngắn
                continue # Chuyển sang frame tiếp theo

            # Tạo ảnh overlay trong suốt (RGBA) cùng kích thước với frame nhận được
            # Kích thước có thể thay đổi nếu camera trả về độ phân giải khác 640x480
            h_img, w_img = img_bgr.shape[:2]
            bbox_overlay_rgba = np.zeros((h_img, w_img, 4), dtype=np.uint8)

            # Chuyển sang ảnh xám để tăng tốc độ nhận diện
            gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
            gray = cv2.equalizeHist(gray) # Cải thiện độ tương phản có thể giúp nhận diện

            # Nhận diện khuôn mặt
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=FACE_SCALE_FACTOR,
                minNeighbors=FACE_MIN_NEIGHBORS,
                minSize=FACE_MIN_SIZE
            )

            num_faces_detected = len(faces)
            face_detected_this_frame = (num_faces_detected > 0)
            can_save_this_frame = face_detected_this_frame and (frame_count >= last_save_frame + SAVE_INTERVAL)

            if face_detected_this_frame:
                label_html = f"Phát hiện {num_faces_detected} khuôn mặt."
                if can_save_this_frame:
                    label_html += " (Đang lưu...)"
            else:
                label_html = "Không phát hiện khuôn mặt."

            # Xử lý từng khuôn mặt được phát hiện
            for (x, y, w, h) in faces:
                # 1. Vẽ hình chữ nhật quanh khuôn mặt lên overlay
                cv2.rectangle(bbox_overlay_rgba, (x, y), (x + w, y + h), (255, 0, 0, 255), 2) # Màu xanh dương, dày 2px

                # 2. Lưu ảnh khuôn mặt (nếu đến lượt lưu)
                if can_save_this_frame:
                    # Cắt vùng khuôn mặt từ ảnh gốc (ảnh màu BGR)
                    face_roi = img_bgr[y:y+h, x:x+w]

                    # Đảm bảo vùng ROI không rỗng và hợp lệ trước khi lưu
                    if face_roi.size > 0:
                        # Tạo tên file duy nhất (timestamp + counter)
                        timestamp_ms = int(time.time() * 1000)
                        filename = os.path.join(SAVE_FOLDER_PATH, f"face_{timestamp_ms}_{saved_face_count:04d}.jpg")
                        try:
                            # Lưu file ảnh khuôn mặt chất lượng cao
                            cv2.imwrite(filename, face_roi, [cv2.IMWRITE_JPEG_QUALITY, 95])
                            # print(f"-> Đã lưu: {filename}") # Bỏ comment nếu muốn thấy từng file được lưu
                            saved_face_count += 1
                        except Exception as e_save:
                            print(f"LỖI khi lưu file {filename}: {e_save}")
                    else:
                        print(f"Cảnh báo: Bỏ qua lưu khuôn mặt rỗng tại frame {frame_count}.")

                # 3. Nhận diện mắt trong vùng khuôn mặt (ROI)
                roi_gray_face = gray[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(
                    roi_gray_face,
                    scaleFactor=EYE_SCALE_FACTOR,
                    minNeighbors=EYE_MIN_NEIGHBORS,
                    minSize=EYE_MIN_SIZE
                )
                # Vẽ hình chữ nhật quanh mắt (trên overlay, tọa độ tương đối với ảnh gốc)
                for (ex, ey, ew, eh) in eyes:
                    cv2.rectangle(bbox_overlay_rgba, (x + ex, y + ey), (x + ex + ew, y + ey + eh), (0, 255, 0, 255), 1) # Màu xanh lá, dày 1px

            # Cập nhật last_save_frame nếu đã lưu trong frame này
            if can_save_this_frame:
                last_save_frame = frame_count

            # Đặt kênh Alpha của overlay thành 255 (không trong suốt) chỉ ở những nơi có vẽ
            # Điều này làm cho phần còn lại của overlay hoàn toàn trong suốt
            alpha_channel = bbox_overlay_rgba[:, :, 3]
            alpha_channel[np.any(bbox_overlay_rgba[:, :, :3] > 0, axis=2)] = 255

            # Chuyển đổi overlay (RGBA) sang định dạng data URL (PNG) để gửi cho JS
            bbox_data_url = bbox_to_bytes(bbox_overlay_rgba)

            # In thông tin frame (tùy chọn)
            # print(f"Frame {frame_count+1}/{MAX_FRAMES_TO_PROCESS}: {label_html} | Faces: {num_faces_detected}")
            # print(f"    Timings (ms): Create: {js_reply.get('create',0)}, Show: {js_reply.get('show',0)}, Capture: {js_reply.get('capture',0)}")

            frame_count += 1
            time.sleep(0.01) # Thêm một độ trễ nhỏ để tránh quá tải CPU/JS

        # Hết vòng lặp (hoặc do max_frames hoặc do dừng thủ công)
        if frame_count >= MAX_FRAMES_TO_PROCESS:
            print(f"\nĐã đạt số khung hình tối đa ({MAX_FRAMES_TO_PROCESS}). Dừng xử lý.")

    except KeyboardInterrupt:
        # Xử lý khi người dùng ngắt bằng Ctrl+C trong Colab output
        print("\nĐã nhận tín hiệu dừng từ bàn phím (KeyboardInterrupt).")
    except Exception as e:
        # Bắt các lỗi không mong muốn khác trong vòng lặp Python
        print(f"\nĐã xảy ra lỗi không mong muốn trong vòng lặp chính: {e}")
        import traceback
        traceback.print_exc() # In chi tiết lỗi để gỡ lỗi
    finally:
        # Dọn dẹp JavaScript DOM bất kể vòng lặp kết thúc như thế nào
        print("\n--- Dọn dẹp và Kết thúc ---")
        try:
            # Gọi hàm JS để xóa các phần tử video, canvas, etc.
            eval_js('removeDom()')
            print("Đã gửi yêu cầu dọn dẹp giao diện JavaScript.")
        except Exception as e_cleanup:
            # Lỗi này có thể xảy ra nếu môi trường JS đã bị hủy hoặc có vấn đề khác
            print(f"Lưu ý: Có thể đã xảy ra lỗi khi cố gắng dọn dẹp JavaScript DOM: {e_cleanup}")
            print("Điều này thường không nghiêm trọng nếu script đã kết thúc.")

        # In tóm tắt kết quả
        print(f"\nĐã xử lý tổng cộng {frame_count} khung hình.")
        print(f"Đã lưu {saved_face_count} ảnh khuôn mặt vào thư mục: '{SAVE_FOLDER_PATH}'")
        print("Chương trình đã kết thúc.")

# --- 8. Run the main function ---
if __name__ == "__main__":
    run_face_detection_and_save()

--- Khởi tạo ---
Sử dụng thư mục lưu trữ hiện có: /content/khuon_mat_da_luu_colab

Đang kiểm tra và tải các tệp Haar Cascade...
haarcascade_frontalface_default.xml đã tồn tại.
-> Bộ phân loại khuôn mặt đã tải thành công.
haarcascade_eye.xml đã tồn tại.
-> Bộ phân loại mắt đã tải thành công.

--- Bắt đầu Luồng Video và Nhận diện ---
Trình duyệt sẽ yêu cầu quyền truy cập camera. Vui lòng đồng ý.


<IPython.core.display.Javascript object>


Đã nhận tín hiệu dừng từ bàn phím (KeyboardInterrupt).

--- Dọn dẹp và Kết thúc ---
Đã gửi yêu cầu dọn dẹp giao diện JavaScript.

Đã xử lý tổng cộng 492 khung hình.
Đã lưu 15 ảnh khuôn mặt vào thư mục: '/content/khuon_mat_da_luu_colab'
Chương trình đã kết thúc.


II. xử lý ảnh biển số xe

In [ ]:
from PIL import Image
import cv2
import torch
import math
from IPython.display import display
import os


In [ ]:
import numpy as np
import math
import cv2

def changeContrast(img):
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)
    limg = cv2.merge((cl,a,b))
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return enhanced_img

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def compute_skew(src_img, center_thres):
    if len(src_img.shape) == 3:
        h, w, _ = src_img.shape
    elif len(src_img.shape) == 2:
        h, w = src_img.shape
    else:
        print('upsupported image type')
    img = cv2.medianBlur(src_img, 3)
    edges = cv2.Canny(img,  threshold1 = 30,  threshold2 = 100, apertureSize = 3, L2gradient = True)
    lines = cv2.HoughLinesP(edges, 1, math.pi/180, 30, minLineLength=w / 1.5, maxLineGap=h/3.0)
    if lines is None:
        return 1

    min_line = 100
    min_line_pos = 0
    for i in range (len(lines)):
        for x1, y1, x2, y2 in lines[i]:
            center_point = [((x1+x2)/2), ((y1+y2)/2)]
            if center_thres == 1:
                if center_point[1] < 7:
                    continue
            if center_point[1] < min_line:
                min_line = center_point[1]
                min_line_pos = i

    angle = 0.0
    nlines = lines.size
    cnt = 0
    for x1, y1, x2, y2 in lines[min_line_pos]:
        ang = np.arctan2(y2 - y1, x2 - x1)
        if math.fabs(ang) <= 30: # excluding extreme rotations
            angle += ang
            cnt += 1
    if cnt == 0:
        return 0.0
    return (angle / cnt)*180/math.pi

def deskew(src_img, change_cons, center_thres):
    if change_cons == 1:
        return rotate_image(src_img, compute_skew(changeContrast(src_img), center_thres))
    else:
        return rotate_image(src_img, compute_skew(src_img, center_thres))


In [ ]:
import math

# license plate type classification helper function
def linear_equation(x1, y1, x2, y2):
    b = y1 - (y2 - y1) * x1 / (x2 - x1)
    a = (y1 - b) / x1
    return a, b

def check_point_linear(x, y, x1, y1, x2, y2):
    a, b = linear_equation(x1, y1, x2, y2)
    y_pred = a*x+b
    return(math.isclose(y_pred, y, abs_tol = 3))

# detect character and number in license plate
def read_plate(yolo_license_plate, im):
    LP_type = "1"
    results = yolo_license_plate(im)
    bb_list = results.pandas().xyxy[0].values.tolist()
    if len(bb_list) == 0 or len(bb_list) < 7 or len(bb_list) > 10:
        return "unknown"
    center_list = []
    y_mean = 0
    y_sum = 0
    for bb in bb_list:
        x_c = (bb[0]+bb[2])/2
        y_c = (bb[1]+bb[3])/2
        y_sum += y_c
        center_list.append([x_c,y_c,bb[-1]])

    # find 2 point to draw line
    l_point = center_list[0]
    r_point = center_list[0]
    for cp in center_list:
        if cp[0] < l_point[0]:
            l_point = cp
        if cp[0] > r_point[0]:
            r_point = cp
    for ct in center_list:
        if l_point[0] != r_point[0]:
            if (check_point_linear(ct[0], ct[1], l_point[0], l_point[1], r_point[0], r_point[1]) == False):
                LP_type = "2"

    y_mean = int(int(y_sum) / len(bb_list))
    size = results.pandas().s

    # 1 line plates and 2 line plates
    line_1 = []
    line_2 = []
    license_plate = ""
    if LP_type == "2":
        for c in center_list:
            if int(c[1]) > y_mean:
                line_2.append(c)
            else:
                line_1.append(c)
        for l1 in sorted(line_1, key = lambda x: x[0]):
            license_plate += str(l1[2])
        license_plate += "-"
        for l2 in sorted(line_2, key = lambda x: x[0]):
            license_plate += str(l2[2])
    else:
        for l in sorted(center_list, key = lambda x: x[0]):
            license_plate += str(l[2])
    return license_plate

In [ ]:
import torch
import os # Thêm thư viện os để kiểm tra đường dẫn

# --- Bước 1: Kết nối Google Drive (Chạy ô này trước) ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive đã được kết nối.")
except ImportError:
    print("Không chạy trên Google Colab hoặc không tìm thấy thư viện google.colab.")
    # Nếu không chạy trên Colab, bạn cần đảm bảo đường dẫn bên dưới là đúng với máy cục bộ
    pass

# --- Bước 2: Xác định đường dẫn chính xác ---
# !!! QUAN TRỌNG: Hãy kiểm tra và sửa lại các đường dẫn này cho đúng với cấu trúc thư mục của bạn trên Drive !!!

# Đường dẫn đến thư mục gốc của dự án trên Drive
base_drive_path = '/content/drive/My Drive/BTL cuối kỳ AI/'

# Đường dẫn ĐẦY ĐỦ đến thư mục chứa mã nguồn YOLO (ví dụ: yolov6)
# **Quan trọng**: Đây phải là thư mục chứa file hubconf.py (hoặc tương đương của yolov6)
yolo_code_repo_path = os.path.join(base_drive_path, 'yolov5') # Hoặc tên thư mục đúng

# Đường dẫn ĐẦY ĐỦ đến file model detector
model_path_detector = os.path.join(base_drive_path, 'model/LP_detector.pt') # Sửa 'model/LP_detector.pt' nếu cần

# Đường dẫn ĐẦY ĐỦ đến file model OCR
model_path_ocr = os.path.join(base_drive_path, 'model/LP_ocr.pt') # Sửa 'model/LP_ocr.pt' nếu cần

# --- Kiểm tra sự tồn tại của đường dẫn (Tùy chọn nhưng hữu ích) ---
print(f"Kiểm tra đường dẫn repo YOLO: {yolo_code_repo_path}")
if not os.path.isdir(yolo_code_repo_path):
    print(f"LỖI: Không tìm thấy thư mục repo YOLO tại '{yolo_code_repo_path}'. Hãy kiểm tra lại đường dẫn.")
elif not os.path.exists(os.path.join(yolo_code_repo_path, 'hubconf.py')):
     # Lưu ý: YOLOv6 có thể dùng cơ chế khác, không phải lúc nào cũng là hubconf.py
     print(f"CẢNH BÁO: Không tìm thấy file 'hubconf.py' trong '{yolo_code_repo_path}'. Torch Hub có thể không hoạt động nếu repo này không hỗ trợ.")

print(f"Kiểm tra đường dẫn model Detector: {model_path_detector}")
if not os.path.exists(model_path_detector):
    print(f"LỖI: Không tìm thấy file model Detector tại '{model_path_detector}'. Hãy kiểm tra lại đường dẫn.")

print(f"Kiểm tra đường dẫn model OCR: {model_path_ocr}")
if not os.path.exists(model_path_ocr):
    print(f"LỖI: Không tìm thấy file model OCR tại '{model_path_ocr}'. Hãy kiểm tra lại đường dẫn.")

# --- Bước 3: Load model với đường dẫn đã sửa ---
try:
    print("Đang thử tải models...")
    # Sử dụng đường dẫn ĐẦY ĐỦ đã xác định ở trên
    # Argument đầu tiên là đường dẫn đến repo code (nơi có hubconf.py)
    yolo_LP_detect = torch.hub.load(yolo_code_repo_path, 'custom', path=model_path_detector, force_reload=True, source='local')
    yolo_license_plate = torch.hub.load(yolo_code_repo_path, 'custom', path=model_path_ocr, force_reload=True, source='local')

    print("Tải model thành công!")

    # set model confidence threshold
    # yolo_LP_detect.conf = 0.6
    yolo_license_plate.conf = 0.60

except FileNotFoundError as e:
    print(f"\nLỖI FileNotFoundError khi tải model: {e}")
    print("-> Vui lòng kiểm tra kỹ các đường dẫn đã khai báo ở Bước 2 có chính xác không.")
    print(f"-> Đảm bảo thư mục '{yolo_code_repo_path}' tồn tại và chứa mã nguồn YOLO cần thiết (bao gồm cả file như hubconf.py nếu Torch Hub yêu cầu).")
    print(f"-> Đảm bảo các file model tại '{model_path_detector}' và '{model_path_ocr}' tồn tại.")
except Exception as e:
    print(f"\nĐã xảy ra lỗi khác khi tải model: {e}")
    print("-> Có thể repo YOLO bạn đang dùng không tương thích với torch.hub.load hoặc model không tương thích với code.")

ValueError: mount failed

In [ ]:
# load yolo model for detect and character detection stage
# please download yolov5 from our link on github
yolo_LP_detect = torch.hub.load('yolov5', 'custom', path='model/LP_detector.pt', force_reload=True, source='local')
yolo_license_plate = torch.hub.load('yolov5', 'custom', path='model/LP_ocr.pt', force_reload=True, source='local')

# set model confidence threshold
# yolo_LP_detect.conf = 0.6
yolo_license_plate.conf = 0.60

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov5/hubconf.py'

In [ ]:
#enter image path here
img_file = "test_image/119.jpg"
img = cv2.imread(img_file)
plates = yolo_LP_detect(img, size=640)

In [ ]:

list_plates = plates.pandas().xyxy[0].values.tolist()
list_read_plates = set()
count = 0
if len(list_plates) == 0:
    lp = read_plate(yolo_license_plate,img)
    if lp != "unknown":
        list_read_plates.add(lp)
else:
    for plate in list_plates:
        flag = 0
        x = int(plate[0]) # xmin
        y = int(plate[1]) # ymin
        w = int(plate[2] - plate[0]) # xmax - xmin
        h = int(plate[3] - plate[1]) # ymax - ymin
        crop_img = img[y:y+h, x:x+w]
        cv2.rectangle(img, (int(plate[0]),int(plate[1])), (int(plate[2]),int(plate[3])), color = (0,0,225), thickness = 2)
        cv2.imwrite("crop.jpg", crop_img)
        rc_image = cv2.imread("crop.jpg")
        lp = ""
        count+=1
        for cc in range(0,2):
            for ct in range(0,2):
                lp = helper.read_plate(yolo_license_plate, utils_rotate.deskew(crop_img, cc, ct))
                if lp != "unknown":
                    list_read_plates.add(lp)
                    flag = 1
                    break
            if flag == 1:
                break

In [ ]:
print(list_read_plates)
img = Image.open(img_file)
basewidth = 500
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth,hsize), Image.ANTIALIAS)
display(img)